In [1]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
pip install dask

In [2]:
#Импорт библиотек
import dask.dataframe as dd
import pandas as pd
import json

In [3]:
#Запись файлов
attr = dd.read_parquet('/kaggle/input/attributes/attributes.parquet')
train = dd.read_parquet('/kaggle/input/train-data/train.parquet')

#Смена типа данных для сокращения требуемого объёма памяти
attr['variantid'] = attr['variantid'].astype('int32')
train['variantid1'] = train['variantid1'].astype('int32')
train['variantid2'] = train['variantid2'].astype('int32')

#Деление на классы и формирование сбалансированной выборки
class_0 = train[train['target'] == 0]
class_1 = train[train['target'] == 1]

n_class_0 = min(2500, class_0.shape[0].compute())
n_class_1 = min(2500, class_1.shape[0].compute())
n = train.shape[0].compute()

del train

sample_class_0 = class_0.sample(frac=n_class_0/n, random_state=42)
sample_class_1 = class_1.sample(frac=n_class_1/n, random_state=42)

balanced_sample = dd.concat([sample_class_0, sample_class_1])

#Выбор необходимых значений из attr
variant_ids = dd.concat([balanced_sample['variantid1'], balanced_sample['variantid2']]).unique()
variant_ids = variant_ids.compute().values
attr = attr[attr['variantid'].isin(variant_ids)]

При анализе данных, представленных в файле attributes.parquet, было выявлено, что в столбце characteristic_attributes_mapping содержится 6016 атрибутов с различными названиями, однако некоторые из них несут одинаковый смысл, несмотря на отличия в формулировке (после попытки объединения осталось порядка 4000 атрибутов, поэтому было принято решение выделить наиболее часто встречающиеся из них в отдельные столбцы - цвет, бренд, тип и страну, а оставшиеся векторизовать).

In [4]:
#Преобразование строки JSON в объект Python
attr['attributes_dicts'] = attr['characteristic_attributes_mapping'].map_partitions(lambda x: x.apply(json.loads), meta=('x', 'object'))

#Выделение наиболее часто встречающихся атрибутов в отдельные столбцы
def extract_attributes(attrs):
    color = None
    brand = None
    type_ = None
    country = None
    for key in attrs.keys():
        if 'Цвет' in key:
            color = attrs.get(key)[0].lower()
        if 'Бренд' in key or 'Издательство' in key:
            brand = attrs.get(key)[0].lower()
        if 'Тип' in key:
            type_ = attrs.get(key)[0].lower()
        if 'Страна' in key or 'Изготовитель' in key:
            country = attrs.get(key)[0]
    return pd.Series({'color': color, 'brand': brand, 'type': type_, 'country': country})

attr[['color', 'brand', 'type', 'country']] = attr['attributes_dicts'].map_partitions(lambda x: x.apply(extract_attributes), meta={'color': 'object', 'brand': 'object', 'type': 'object', 'country': 'object'})

#Вынесение в отдельные столбцы информации о категориях 2 и 4 уровня
attr['cat2'] = attr['categories'].map_partitions(lambda x: x.apply(lambda y: json.loads(y).get('2')), meta=('x', 'object'))
attr['cat4'] = attr['categories'].map_partitions(lambda x: x.apply(lambda y: json.loads(y).get('4')), meta=('x', 'object'))
attr = attr.drop(['categories'], axis = 1)

#Запись в переменные файлов с эмбеддингами
emb = dd.read_parquet('/kaggle/input/resnet50embeddings/resnet.parquet', columns = ['variantid', 'main_pic_embeddings_resnet_v1'])
text = dd.read_parquet('/kaggle/input/text-and-bert/text_and_bert.parquet', columns = ['variantid', 'description', 'name_bert_64'])

#Смена типа данных для сокращения требуемого объёма памяти
emb['variantid'] = emb['variantid'].astype('int32')
text['variantid'] = text['variantid'].astype('int32')

#Добавление новой информации к данным о категориях и атрибутах
attr = dd.merge(attr, emb, on='variantid', how='left')
del emb
attr = dd.merge(attr, text, on='variantid', how='left')
del text

Нижепредставленный закомментированный код изначально планировалось использовать для выделения нескольких дополнительных признаков для обучения модели, например количества одинаковых атрибутов у двух сравниваемых товаров, однако впоследствии было решено сравнивать лишь эмбединги, полученные из строки атрибутов.

In [ ]:
'''
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Определение всех возможных названий атрибутов товаров для дальнейшего объединения схожих
unique_attrib = set()
for dict_ in attr['attributes_dicts']:
    unique_attrib.update(dict_.keys())
    
unique_attrib = list(unique_attrib)

#Создание словаря сопоставления для объединения наименований схожих атрибутов через векторизацию и расчёт косинусного расстояния
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(unique_attrib)

threshold = 0.8
cosine_sim = cosine_similarity(tfidf_matrix)

attribute_mapping = {}
for i in range(len(cosine_sim)):
    for j in range(i + 1, len(cosine_sim)):
        if cosine_sim[i][j] > threshold:
            attribute_mapping.setdefault(unique_attrib[i], []).append(unique_attrib[j])

#Создание обратного сопоставления для замены
reverse_mapping = {}
for key, values in attribute_mapping.items():
    for value in values:
        reverse_mapping[value] = key
        
#Замена значений в датафрейме
def replace_attribute_keys(attribute_dict):
    return {reverse_mapping.get(key, key): value for key, value in attribute_dict.items()}

attr['attributes_dicts'] = attr['attributes_dicts'].map_partitions(lambda x: x.apply(replace_attribute_keys, meta=('x', 'object')))

#Освобождение памяти
del attribute_mapping
del reverse_mapping
gc.collect()
'''

In [4]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
from torch import nn
from torch.nn import init
from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass

In [7]:
#Код для получения эмбеддингов из строк
class ProdFeatureEncoder(nn.Module):
    """
    Model for creating embeddings with pre-trained ruBERT-tiny BERT.

    Attributes:
        config (object): Configuration object containing model hyperparameters.
        tokenizer (AutoTokenizer): Tokenizer instance for ruBERT-tiny.
        model (AutoModel): Pre-trained ruBERT-tiny model instance.
        fc (nn.Linear): Linear layer for dimensionality reduction.
    """
    def __init__(self, config):
        """
        Initializes the ProdFeatureEncoder model.

        Args:
            config (object): Configuration object containing model hyperparameters.
        """
        super().__init__()
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/rubert-tiny")
        self.model = AutoModel.from_pretrained("/kaggle/input/rubert-tiny")
        self.fc = nn.Linear(self.config.bert_output_size, self.config.embedding_size)
        init.xavier_uniform_(self.fc.weight)
        self.norm = nn.LayerNorm(self.config.embedding_size)

    def forward(self, text: str):
        """
        Creates an embedding for the input text.
        Args:
            text (str): Input text to create an embedding for.
        Returns:
            torch.Tensor: Embedding vector for the input text.
        """
        tokens = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        model_output = self.model(**{k: v.to(self.model.device) for k, v in tokens.items()})
        embedding = model_output.last_hidden_state[:, 0, :]
        embedding = self.fc(embedding)
        return embedding[0]

@dataclass
class ModelConfig:
    bert_output_size = 312
    embedding_size = 128

def encodering(line):
    encoded_input=encoder(line)
    return encoded_input.clone().detach().numpy()

def apply_encodering(series):
    return series.map(lambda x: encodering(x) if x is not None and not pd.isna(x) else None, meta=('x', 'object'))

In [8]:
model_config = ModelConfig()
encoder = ProdFeatureEncoder(model_config)

In [9]:
attr['characteristic_attributes_mapping'] = apply_encodering(attr['characteristic_attributes_mapping'])
attr['color'] = apply_encodering(attr['color'])
attr['type'] = apply_encodering(attr['type'])
attr['cat2'] = apply_encodering(attr['cat2'])
attr['cat4'] = apply_encodering(attr['cat4'])
attr['description'] = apply_encodering(attr['description'])

In [10]:
from scipy.spatial.distance import cosine

#Объединение с тренировочным датафреймом
merged_df = dd.merge(balanced_sample, attr.add_suffix('1'), on='variantid1', how='inner')
merged_df = dd.merge(merged_df, attr.add_suffix('2'), on='variantid2', how='inner')

'''
#Определение количества одинаковых наименований атрибутов для пар товаров
def count_common_keys(row):
    keys1 = row['attributes_dicts1'].keys()
    keys2 = row['attributes_dicts2'].keys()
    return len(set(keys1) & set(keys2))

merged_df['count_of_eq_attr'] = merged_df.apply(count_common_keys, axis=1)
'''

#Получение столбцов со сравнением признаков пары товаров
merged_df['brand_eq'] = (merged_df['brand1'] == merged_df['brand2']).astype(int)
merged_df['country_eq'] = (merged_df['country1'] == merged_df['country2']).astype(int)

#Функция для вычисления косинусного расстояния
def compute_cosine_distance(v1, v2):
    if v1 is None or v2 is None:
        return None
    return cosine(v1, v2)

#Функция для расчета манхэттенского расстояния
def manhattan_distance(vec1, vec2):
    return sum(abs(a - b) for a, b in zip(vec1, vec2))

#Расчёт косинусных расстояний
merged_df['attr_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: compute_cosine_distance(row['characteristic_attributes_mapping1'], row['characteristic_attributes_mapping2']), axis=1), meta=('x', 'f8'))
merged_df['color_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: compute_cosine_distance(row['color1'], row['color2']), axis=1), meta=('x', 'f8'))
merged_df['type_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: compute_cosine_distance(row['type1'], row['type2']), axis=1), meta=('x', 'f8'))
merged_df['cat2_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: compute_cosine_distance(row['cat21'], row['cat22']), axis=1), meta=('x', 'f8'))
merged_df['cat4_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: compute_cosine_distance(row['cat41'], row['cat42']), axis=1), meta=('x', 'f8'))
merged_df['pic_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: manhattan_distance(row['main_pic_embeddings_resnet_v11'][0], row['main_pic_embeddings_resnet_v12'][0]), axis=1), meta=('x', 'f8'))
merged_df['description_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: compute_cosine_distance(row['description1'], row['description2']), axis=1), meta=('x', 'f8'))
merged_df['name_dis'] = merged_df.map_partitions(lambda df: df.apply(lambda row: compute_cosine_distance(row['name_bert_641'], row['name_bert_642']), axis=1), meta=('x', 'f8'))

#Удаление ненужных столбцов
columns_to_drop = ['variantid1', 'variantid2', 'brand1', 'brand2', 'country1', 'country2', 'attributes_dicts1', 'attributes_dicts2']

merged_df = merged_df.drop(columns=columns_to_drop)

merged_df = merged_df.persist()
merged_df = merged_df.compute()

merged_df.to_parquet('/kaggle/working/merged_df.parquet')